# 채권분석리포트 전처리

1. pdf불러와서 fitz 라이브러리를 활용하여 text추출

In [ ]:
date=[]
contents=[]
str=""


import fitz
import os
os.chdir(r'C:/Users/HanBi/Desktop/2021/project1/data/bonds/')
from glob import glob
pdfs = glob('*.pdf')
for pdf in pdfs:
    doc = fitz.open(r'C:/Users/HanBi/Desktop/2021/project1/data/bonds/'+pdf) #for문 하나씩 추가
    page_count = doc.pageCount
    page=0
    while(page<page_count):
        p = doc.loadPage(page)
        str = str+p.getText()
        print(p.getText())
        page +=1
    date.append(pdf[0:8])
    contents.append(str)
    

2. 정규식, 수작업 제거 함수를 통한 불용어 제거

In [ ]:
import pandas as pd
df=pd.read_csv(r"C:\Users\HanBi\Desktop\2021\project1\crawling\채권분석리포트.csv")  

content=df['content'].tolist()

import re

content2=[]
for i in content:
    p=re.sub('자료: \w.+', '', str(i))
    content2.append(p)
    
content3=[]
for i in content2:
    p=re.sub(' +',' ', str(i))
    content3.append(p)

content4=[]
for i in content3:
    p=re.sub('\d{2,3}-\d{3,4}-\d{4}','',str(i))
    content4.append(p)

content5=[]
for i in content4:
    p=re.sub('[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+','',str(i))
    content5.append(p)

content6=[]
for i in content5:
    p=re.sub(' ([\d\. ])+[A-Za-z]{3}','',str(i))
    content6.append(p)
    
def delete_string(text, dele): #text->content list # dele->지워야할 리스트
    fixed_text=[]
    for i in text:
        for j in dele:
            i=i.replace(j,"")
        fixed_text.append(i)
    return fixed_text

del_list=['본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정보를 바탕으로 작성한 것이나 당사는 그 정확성이나 완전성을 보장할 수 없으며, 과거의 자료를 기초로 한 투자참고 자료로서 향후 주가 움직임은 과거의 패턴과 다를 수 있습니다. 고객께서는 자신의 판단과 책임 하에 종목 선택이나 투자시기에 대해 최종 결정하시기 바라며, 본 자료는 어떠한 경우에도 고객의 증권투자 결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니다.',
          'Daewoo Daewoo Fixed Income Daily 채권 윤일광()',
          'Daewoo 채권 윤여삼 ()','IBKS Bond Daily Brief','본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정보를 바탕으로 작성한 것이나 당사는 그 정확성이나 완전성을 보장할 수 없으며, 과거의 자료를 기초로 한 투자참고 자료로서 향후 주가 움직임은 과거의 패턴과 다를 수 있습니다. 고객께서는 자신 의 판단과 책임 하에 종목 선택이나 투자시기에 대해 최종 결정하시기 바라며, 본 자료는 어떠한 경우에도 고객의 증권투자 결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니다. 동 자료에 게재된 내용들은 외부의 압력이나 부당한 간섭 없이 본인의 의 견을 정확하게 반영하여 작성되었음을 확인합니다. 동 자료는 기관투자가 또는 제3자에게 사전 제공한 사실이 없습니다. 동 자료는 조사분석자료 작성에 참여한 외부인(계열회사 및 그 임직원등)이 없습니다.',
          'Fixed Income Strategist','오 창 섭 (, 6915-5682)', '주식시장 동향',
          '본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정보를 바탕으로 작성한 것이나 당사는 그 정확성이나 완전성을 보장할 수 없으며, 과거의 자료를 기초로 한 투자참고 자료로서 향후 주가 움직임은 과거의 패턴과 다를 수 있습니다. 고객께서는 자신의 판단과 책임 하에 종목 선택이나 투자시기에 대해 최종 결정하시기 바라며, 본 자료는 어떠한 경우 에도 고객의 증권투자 결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니다.',
          'HMC Investment Securities Research Centerd Market View BOND MARKET WEEKLY',
          'Fixed Income & Credit','민동원()','메리츠증권','리서치센터itz','Bond Daily',
          '금융투자협회 본 자료에 게재된 내용들은 본인의 의견을 정확하게 반영하고 있으며, 외부의 부당한 압력이나 간섭없이 작성되었음을 확인합니다. (작성자: 민동원) 동 자료는 투자자들의 투자판단에 참고가 되는 정보제공을 목적으로 배포되는 자료입니다. 동 자료에 수록된 내용은 당사 리서치센터의 추정치로서 오차가 발생할 수 있으며 정확성이나 완벽성은 보장하지 않습니다. 동 자료를 이용하시는 분은 동 자료와 관련한 투자의 최종 결정은 자신의 판단으로 하시기 바랍니다.',
          'www.hmcib.comed Income _ strategy Forward Solution HMC Securities',
          'Fixed Income Weekly','Fixed Income Outlook www.ibks.com','Fixed Income',
          '노 호 영 (, 6915-5681)','Bond Market Monthlyed Income : 이정준 Tel : 02)3787-2627 E-mail :',
          'HMC Investment Securities Research Centerket Issue','본 자료는 투자자의 증권투자를 돕기 위하여 작성된 당사의 저작물입니다. 당사 고객에 한하여 배포되는 자료로서 어떠한 경우에도 복사, 배포, 전송, 변형, 대여될 수 없습니다. 본 조사자료에 수록된 내용은 당사 리서치센터가 신뢰할 만한 자료 및 정보로부터 얻어진 것이나, 당사는 그 정확성이나 안전성을 보장할 수 없습니다. 따라서 어떠한 경우에도 본 자료는 고객의 증권투자의 결과에 대한 법적 책임소재에 대한 증빙자료 로 사용할 수 없습니다.',
          '유재호 02) 3787-5283','이화진 02) 3787-4711','이소원 02) 3787-4952',
          '동 자료에 게시된 내용들은 본인의 의견을 정확하게 반영하고 있으며, 외부의 부당한 압력이나 간섭 없이 작성되었음을 확인합니다 본 자료는 투자자의 증권투자를 돕기 위하여 당사 고객에 한하여 배포되는 자료로서 어떠한 경우에도 복사되거나 대여 될 수 없습니다. 본 조사자료에 수록된 내용은 당사 리서치센터가 신뢰할 만한 자료 및 정보로부터 얻어진 것이나, 당사는 그 정확성이나 안전성을 보장할 수 없습니다. 따라서, 어떠한 경우에도 본 자료는 고객의 증권투자의 결과에 대한 법적 책임소재에 대한 증빙자료로 사용할 수 없습니다.',
          'Economist 정용택 ☎ 02) 2184-2321','R.A. 정성욱 ☎ 02) 2184-2368','FI Strategist',
          'www.imeritz.com','Bond Market Comment','Daewoo 채권 윤여삼()','Weekly Bond Market Weekly Bond Market',
          '키움증권 리서치센터','투자전략팀','정성욱 ☎ 02) 2184-2368',
          'Monthly Bond Market', '이현석 ☎ 02) 2184-2346','Weekly Credit Bond',
          'Income Analyst 오창섭','주: 발표문의 중요내용 변화에 밑줄 표시,',
          'www.hmcib.com','dit : 황원하 Tel : 02)3787-2459 E-mail :','채권시장 전망',
          '<그림1>','Weekly ‘Macro & Bond’ Check','ro & Bond Analyst 오창섭',
          '이정준 02) 3787-2627 ','김훈길 769.3023','Credit 손소현 02. 3779-0055',
          'Good Morning 매크로 Call','RA 전병하 02. 3779-8633','유선웅 ☎ 02)6923-7353',
          '▶','▩','채권 이미선   RA 이길영','pliance Notice','본 조사자료는 고객의 투자에 정보를 제공할 목적으로 작성되었으며, 어떠한 경우에도 무단 복제 및 배포 될 수 없습니다. 또한 본 자료에 수록된 내용은 당사가 신뢰할 만한 자료 및 정보로 얻어진 것이나, 그 정확성이나 완전성을 보장할 수 없으므로 투자자 자신의 판단과 책임하에 최종결정을 하시기 바랍니다. 따라서 어떠한 경우에도 본 자료는 고객의 주식투자의 결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니다.',
          '크레딧분석','◎','하나금융투자dit','Analyst 김상만 E-mail : : 3771-7508 RA 김태형 E-mail : : 3771-7743',
          'LIG Research Center','김민규 02) 3787-5298',' Bond Comment','Bond Macro Analysis',
          'Weekly Bond Market',  'Market Analyst',  'RA',  'Weekly Bond Market','Credit Analyst',
          '임채민 ☎ 02) 2184-2201','R.A.','정성욱 02) 2184-2368','www.ibks.com',
          '본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정보를 바탕으로 작성한 것이나 당사는 그 정확성이나 완전성을 보장할 수 없으며, 과거의 자료를 기초로 한 투자참고 자료로서 향후 주가 움직 임은 과거의 패턴과 다를 수 있습니다. 고객께서는 자신의 판단과 책임 하에 종목 선택이나 투자시기에 대해 최종 결정하시기 바라며, 본 자료는 어떠한 경우에도 고객의 증권투자 결과에 대한 법적 책임소재 의 증빙자료로 사용될 수 없습니다.',
          'Economic.deri.co.kr Fundflows Weekly Asset Market Brief','Credit Analyst 김민정 ☎ 2184-2377',
          '★','Credit 손소현','전병하','02. 3779-0055','02. 3779-8633','nghajeon@',
          '나금융투자 2017년 리서치 전망 포럼','이베스트투자증권 리서치센터','Income Weekly Part I',
          'Review & Preview Part II','Preview Part III Weekly Check Point','경제/투자전략',
          '박진영 CPA','02) 3787-2248','ed Income t I',
          '리서치센터 자산분석실 채권전략 이미선','<<Executive Summary>> Review:',
          'Credit Reportdit Weekly','Analyst 최운선 02)6923-7344','ed',
          'Income','Weekly Review','Part III Weekly Check Point',
          '이미선','윤정수','주간','Issue 분석',':','iew'
          '본 조사자료는 고객의 투자에 정보를 제공핛 목적으로 작성되었으며, 어떠핚 경우에도 무단 복제 및 배포 될 수 없습니다. 또핚 본 자료에 수록된 내용은 당사가 싞뢰핛 만핚 자료 및 정보로 얻어짂 것이나, 그 정확성이나 완젂성을 보장핛 수 없으므로 투자자 자싞의 판단과 책임하에 최종결정을 하시기 바랍니다. 따라서 어떠핚 경우에도 본 자료는 고객의 주식투자의 결과에 대핚 법적 책임소재의 증빙자료로 사용될 수 없습니다.',
          'KIWOOM FX DAILY','본 자료는 투자자의 증권투자를 돕기 위하여 당사 고객에 한하여 배포되는 자료로서 어떠한 경우에도 복사되거나 대여 될 수 없습니다. 본 조사자료에 수록된 내용은 당사 리서치센터가 신뢰할 만한 자료 및 정보로부터 얻어진 것이나, 당사는 그 정확성이나 안전성을 보장할 수 없습니다. 따라서, 어떠한 경우에도 본 자료는 고객의 증권투자의 결과에 대한 법적 책임소재에 대한 증빙자료로 사용할 수 없습니다.','[1]',
          '1cutive Summary','•','김상만','신의현','Issue Comment Credit Credit',
          'Com','당사는 동 자료를 기관투자자 또는 제3자에게 사전 제공한 사실이 없습니다.',
          '동 자료에 게시된 내용들은 본인의 의견을 정확하게 반영하고 있으며, 외부의 부당한 압력이나 간섭없이 작성되었음을 확인합니다. 고지사항 본 조사분석자료는 당사의 리서치센터가 신뢰할 수 있는 자료 및 정보로부터 얻은 것이나, 당사가 그 정확성이나 완전성을 보장할 수 없고, 통지 없이 의견이 변경될 수 있습니다.',
          '본 조사분석자료는 유가증권 투자를 위한 정보제공을 목적으로 당사 고객에게 배포되는 참고자료로서, 유가증권의 종류, 종목, 매매의 구분과 방법 등에 관한 의사결정은 전적으로 투자자 자신의 판단과 책임하에 이루어져야 하며, 당사는 본 자료의 내용에 의거하여 행해진 일제의 투자행위 결과에 대하여 어떠한 책임도 지지 않으며 법적 분쟁에서 증거로 사용 될 수 없습니다.',
          '본 조사 분석자료를 무단으로 인용, 복제, 전시, 배포, 전송, 편집, 번역, 출판하는 등의 방법으로 저작권을 침해하는 경우에는 관련법에 의하여 민∙형사상 책임을 지게 됩니다.','채권전략','안예하 02) 3787-5157',
          '김준수 02)3787-3751','nt']    

content6=delete_string(content6,del_list)


# 콜금리 전처리

한달전 date의 콜금리와 비교하여 Labeling 컬럼 추가 (ex)한달전 보다 콜금리가 높다면 up

In [ ]:
import pandas as pd

#크롤링한 콜금리 파일 불러오기
df=pd.read_csv(r"C:\Users\HanBi\Desktop\2021\project1\crawling\콜금리크롤링.csv")

#명칭 기반 인덱싱
df1=df.loc[2:3472]
df2=df.loc[22:3493]

#df2 컬럼네임 바꾸기
newcols = {
    'date': '한달전 date',
    'callrate': '한달전 callrate',
}
df2=df2.rename(columns=newcols) 

#두 데이터프레임 합치기
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df3=pd.concat([df1, df2], axis=1, ignore_index=False)
        
#apply함수써서 labeling 컬럼 만들기
def get_label(df):
    if df['callrate'] > df['한달전 callrate']:
        return "up"
    elif df['callrate'] < df['한달전 callrate']:
        return "down"
    else:
        return "middle"
    
df3['labeling']=df3.apply(get_label,axis=1)